<a href="https://colab.research.google.com/github/Hujaifa98/AIProjects/blob/main/MNIST_Using_GooLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from tensorflow.keras import datasets, layers, models, losses, Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
import matplotlib.pyplot as plt

In [ ]:
(x_train,y_train),(x_test,y_test) = datasets.mnist.load_data()

In [ ]:
x_train = tf.pad(x_train, [[0, 0], [2,2], [2,2]])/255
x_test = tf.pad(x_test, [[0, 0], [2,2], [2,2]])/255
x_train = tf.expand_dims(x_train, axis=3, name=None)
x_test = tf.expand_dims(x_test, axis=3, name=None)
x_train = tf.repeat(x_train, 3, axis=3)
x_test = tf.repeat(x_test, 3, axis=3)
x_val = x_train[-2000:,:,:,:]
y_val = y_train[-2000:]
x_train = x_train[:-2000,:,:,:]
y_train = y_train[:-2000]

In [ ]:
x_train.shape

TensorShape([58000, 32, 32, 3])

In [ ]:
def inception(x,filter_1x1,filter_3x3_reduce,filter_3x3,filter_5x5_reduce,filter_5x5,filter_pool):
  path1 = keras.layers.Conv2D(filter_1x1,(1,1),padding='same',activation='relu')(x)

  path2 = keras.layers.Conv2D(filter_3x3_reduce,(1,1),padding='same',activation='relu')(x)
  path2 = keras.layers.Conv2D(filter_3x3,(3,3),padding='same',activation='relu')(path2)

  path3 = keras.layers.Conv2D(filter_5x5_reduce,(1,1),padding='same',activation='relu')(x)
  path3 = keras.layers.Conv2D(filter_5x5,(5,5),padding='same',activation='relu')(path3)

  path4 = keras.layers.MaxPool2D((3,3),(1,1),padding='same')(x)
  path4 = keras.layers.Conv2D(filter_pool,(1,1),padding='same',activation='relu')(path4)

  return tf.concat([path1,path2,path3,path4] , axis=3)

In [ ]:
inp = layers.Input(shape=(32, 32, 3))
input_tensor = layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=x_train.shape[1:])(inp)
x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(input_tensor)
x = layers.MaxPooling2D(3, strides=2)(x)
x = layers.Conv2D(64, 1, strides=1, padding='same', activation='relu')(x)
x = layers.Conv2D(192, 3, strides=1, padding='same', activation='relu')(x)
x = layers.MaxPooling2D(3, strides=2)(x)
x = inception(x, 64, 96, 128, 16, 32, 32)
x = inception(x, 128, 128, 192, 32, 96, 64)
x = layers.MaxPooling2D(3, strides=2)(x)
x = inception(x, 192, 96, 208, 16, 48, 64)
aux1 = layers.AveragePooling2D((5, 5), strides=3)(x)
aux1 =layers.Conv2D(128, 1, padding='same', activation='relu')(aux1)
aux1 = layers.Flatten()(aux1)
aux1 = layers.Dense(1024, activation='relu')(aux1)
aux1 = layers.Dropout(0.7)(aux1)
aux1 = layers.Dense(10, activation='softmax')(aux1)
x = inception(x, 160, 112, 224, 24, 64, 64)
x = inception(x, 128, 128, 256, 24, 64, 64)
x = inception(x, 112, 144, 288, 32, 64, 64)
aux2 = layers.AveragePooling2D((5, 5), strides=3)(x)
aux2 =layers.Conv2D(128, 1, padding='same', activation='relu')(aux2)
aux2 = layers.Flatten()(aux2)
aux2 = layers.Dense(1024, activation='relu')(aux2)
aux2 = layers.Dropout(0.7)(aux2) 
aux2 = layers.Dense(10, activation='softmax')(aux2)
x = inception(x, 256, 160, 320, 32, 128, 128)
x = layers.MaxPooling2D(3, strides=2)(x)
x = inception(x, 256, 160, 320, 32, 128, 128)
x = inception(x, 384, 192, 384, 48, 128, 128)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
out = layers.Dense(10, activation='softmax')(x)

In [ ]:
model = Model(inputs = inp, outputs = [out, aux1, aux2])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 resizing_4 (Resizing)          (None, 224, 224, 3)  0           ['input_5[0][0]']                
                                                                                                  
 conv2d_50 (Conv2D)             (None, 112, 112, 64  9472        ['resizing_4[0][0]']             
                                )                                                                 
                                                                                                  
 max_pooling2d_15 (MaxPooling2D  (None, 55, 55, 64)  0           ['conv2d_50[0][0]']          

In [ ]:
model.compile(optimizer='adam', 
              loss=[losses.sparse_categorical_crossentropy,
                    losses.sparse_categorical_crossentropy,
                    losses.sparse_categorical_crossentropy],
              loss_weights=[1, 0.3, 0.3],
              metrics=['accuracy'],)
history = model.fit(x_train, [y_train, y_train, y_train], validation_data=(x_val, [y_val, y_val, y_val]), batch_size=64, epochs=40)

Epoch 1/40
  4/907 [..............................] - ETA: 3:40:32 - loss: 3.6740 - dense_8_loss: 2.2944 - dense_5_loss: 2.3007 - dense_7_loss: 2.2980 - dense_8_accuracy: 0.1328 - dense_5_accuracy: 0.0977 - dense_7_accuracy: 0.1406

KeyboardInterrupt: ignored

In [ ]:
x_train = tf.pad(x_train, [[0, 0], [2,2], [2,2]])/255
x_test = tf.pad(x_test, [[0, 0], [2,2], [2,2]])/255
x_train = tf.expand_dims(x_train, axis=3, name=None)
x_test = tf.expand_dims(x_test, axis=3, name=None)
x_train = tf.repeat(x_train, 3, axis=3)
x_test = tf.repeat(x_test, 3, axis=3)
x_val = x_train[-2000:,:,:,:]
y_val = y_train[-2000:]
x_train = x_train[:-2000,:,:,:]
y_train = y_train[:-2000]